In [ ]:
#default_exp unused

# Compute Pillars

## Old Jit Code

#### convert_bounds_to_points

In [ ]:
#export
@jit(nopython=True)
def pib_setup(pillars: np.ndarray, centers: np.ndarray, pcloud: np.ndarray, max_ppp: int, max_pil: int, remove_empty: bool = True):
    """
        a helper method to prepare the actual _get_points_in_bounds method and
        improve the runtime with numba

        param remove_empty: set True if empty pillars should be removed
                            from the final tensor
        returns: numpy array with the pcloud points sorted by pillar. The cloud
                 point cloud tensor is already zero padded / sampled to the correct
                 length. Shape is (pillar_nbr, point_nbr, 4)
    """

    # TODO remove the centers for empty pillars
    pillars_out = np.empty((1, max_ppp, pcloud.shape[1]), dtype=np.float32)
    # mask to keep centers synced
    mask = np.ones(pillars.shape[0], dtype=np.bool_)
    for i in range(pillars.shape[0]):
        pillar_bounds = pillars[i]
        pillar_points = get_points_in_bounds(pillar_bounds, pcloud)
        if remove_empty and pillar_points.shape[0] == 0:
            mask[i] = False
            continue

        if pillar_points.shape[0] < max_ppp:
            pillar_points = zero_pad_array(pillar_points, max_ppp)
        elif pillar_points.shape[0] > max_ppp:
            pillar_points, = sample_array((pillar_points,), max_ppp)

        pillars_out = np.concatenate((pillars_out, np.expand_dims(pillar_points, axis=0)), axis=0)

    centers_np = centers[mask]
    if pillars_out.shape[0] == 0:
        raise ValueError("No pillars calculated.")

    if pillars_out.shape[0] < max_pil:
        pillars_np = zero_pad_array(pillars_out, max_pil)
        centers_np = zero_pad_array(centers, max_pil)
    elif pillars_out.shape[0] > max_pil:
        pillars_np, centers_np = sample_array((pillars_out, centers_np), max_pil)

    return pillars_np, centers_np


@jit(nopython=True)
def get_points_in_bounds(pillar_bounds: np.ndarray, pcloud: np.ndarray):
    """
        The actual point selection
        takes the pillar_bounds for one pillar as np array and returns the pcloud points within those points.

        param pillar_bounds: pillar_bounds for one pillar. Shape must be [[x_min, y_min], [x_max, y_max]]
        returns: 1-D np array with all pcloud points within the given range.

    """
    # TODO: to be tested
    xy_pts = pcloud[:,0:2]
    lb = pillar_bounds[0]
    ub = pillar_bounds[1]
    mask = np.logical_and(lb <= xy_pts, xy_pts <= ub)
    mask = np.logical_and(mask[:,0], mask[:,1])

    return pcloud[mask].astype(np.float32)


In [ ]:
pillar_bounds = np.array([[1,1], [2,2]])
pcloud = np.random.rand(100, 4) * 2
res = get_points_in_bounds(pillar_bounds, pcloud)

pcloud, pcloud.shape, res, res.shape

In [ ]:
pillar_bounds = np.array([[0,0], [1,1]])
pcloud = np.random.rand(100, 4) * 2
time_method(get_points_in_bounds, runs=50, kwargs={"pillar_bounds": pillar_bounds, "pcloud": pcloud})

In [ ]:
pillar_bounds = torch.tensor([[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]],[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]],[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]],[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]]])
pillars_cfg = read_config()['pillars']
pillars = Pillars(pillar_bounds, pillars_cfg)
pillars.convert_bounds_to_points(pc)

pillars.pillars, pillars.pillars.shape, pillars.centers, pillars.centers.shape


In [ ]:
# Does not work because cannot do numerous runs on one object
pillar_bounds = torch.tensor([[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]],[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]],[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]],[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]], [[4, -5], [5, -5]]])
pillars_cfg = read_config()['pillars']
pillars = Pillars(pillar_bounds, pillars_cfg)

time_method(pillars.convert_bounds_to_points, runs=1, kwargs={"pcloud": pc})




# Old pillar class
> Here we can finally define all the classes and methods that do the actual calculation of the pillars. To keep speed
> efficiency, we tried to do without many loops. When there was no way to avoid them, we took advantage of numba



### Pillars class


In [ ]:
#export
class Pillars(torch.utils.data.Dataset):

    def __init__(self, pillar_bounds: torch.Tensor, pcloud: torch.tensor, pillars_cfg: dict):
        if not torch.is_tensor(pillar_bounds):
            raise ValueError("Tensor expected but not given.")

        if pillar_bounds.shape[1] != 2 or pillar_bounds.shape[2] != 2:
            raise ValueError("Wrong tensor dimensions. Expect (2,2)")

        # saves all the pillars
        self.pillar_bounds = pillar_bounds
        self.pillars_cfg = pillars_cfg
        self.pillars = None
        self.centers = None

        #rand_ind = torch.randperm(pcloud.shape[0])
        #self.pcloud = pcloud[rand_ind]
        self.pcloud = pcloud

    def __getitem__(self, item):
        """
            Only iterate over the pillars, centers can be calculated in batch.
            Give sampled/zero-padded tensor in the correct length with the points in the pillar
        """
        pillar = self.pillar_bounds[item]
        pip = Pillars.convert_bounds_to_points(pillar, self.pcloud)
        max_ppp = self.pillars_cfg.getint('max_points_per_pillar')
        """
        pillar_points = torch.zeros((max_ppp, 4), dtype=dt, device=dev)
        n = torch.min(torch.tensor([max_ppp, pip.shape[0]]))

        pillar_points[0:n] = pip[0:n]
        """
        pillar_points = pip
        if pip.shape[0] < max_ppp:
            pillar_points = zero_pad_array(pip, max_ppp)
        elif pip.shape[0] > max_ppp:
            pillar_points, = sample_array((pip,), max_ppp)

        return pillar_points

    def __len__(self):
        return self.pillar_bounds.shape[0]

    @staticmethod
    def convert_bounds_to_points(pillar: torch.tensor, pcloud: torch.tensor):
        """
            Takes its pillar attribute and converts the pillar bounds to points within those bounds.
            Pillar shape must be (2, 2) with the two dimensions being:
            [[x_min, y_min], [x_max, y_max]] for the pillar
            param config: give the whole config parser
            returns: tensor with padded and sampled output in the correct size
        """
        xy_pts = pcloud[:,0:2]
        lb = pillar[0]
        ub = pillar[1]
        mask = torch.logical_and(torch.le(lb, xy_pts), torch.le(xy_pts, ub))
        mask = torch.logical_and(mask[:,0], mask[:,1])

        return pcloud[mask]


    def convert_bounds_to_centers(self):
        """
            converts the pillar bounds into centers.  Pillars center shape must be (pillar_nbr, 2, 2) with the
            last two dimensions being [[x_min, y_min], [x_max, y_max]] for each pillar
        """
        # TODO: Test if method is still running after changing bound array shape

        xy_min = self.pillar_bounds.index_select(dim=1, index=torch.tensor([0], device=dev))
        x_offset = 0.5 * self.pillars_cfg.getfloat('x_step')
        y_offset = 0.5 * self.pillars_cfg.getfloat('y_step')
        offset = torch.tensor([x_offset, y_offset], dtype=dt, device=dev)
        z_center = torch.tensor([(self.pillars_cfg.getfloat('z_max') - self.pillars_cfg.getfloat('z_min')) / 2.0], dtype=dt, device=dev)
        # bring z center on shape from xy_min for concatenation
        z_center = z_center.unsqueeze(0).unsqueeze(0).expand(xy_min.shape[0], -1, -1)

        xy_min = xy_min.add(offset)
        self.centers = torch.cat((xy_min, z_center), dim=2)

In [ ]:
pillar = torch.tensor([[[0, 0], [1,1]]], dtype=dt, device=dev)
pillar_cfg = read_config()['pillars']
pillars_dts = Pillars(pillar, pc, pillar_cfg)
time_method(Pillars.convert_bounds_to_points, runs=1000, kwargs={"pillar": pillar[0], "pcloud": pc})

In [ ]:
pillar_bounds = torch.tensor([[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]]], device=dev)

pillars = Pillars(pillar_bounds, pc, pillar_cfg)

pillars.convert_bounds_to_centers()
pillars.centers

In [ ]:
pillar_bounds = torch.tensor([[[0, 0], [1,1]], [[-1,-1], [0,0]], [[0, -5], [5, 0]]], device=dev)
pillar_cfg = read_config()['pillars']
pillars = Pillars(pillar_bounds, pc, pillar_cfg)

time_method(pillars.convert_bounds_to_centers, runs=1)

### Bring points to same length

##### zero_pad_array()



### Get points inside the pillar

We want to extract the points within specific bounding parameters defined by the pillar they are in.

In [ ]:
#export
def get_points_in_pillar(pcloud, x_min, x_max, y_min, y_max):
    if not torch.is_tensor(pcloud):
        raise ValueError("Tensor expected but not given.")

    if x_max < x_min or y_max < y_min:
        raise ValueError("Unexpected parameters values given. Please check your boundaries.")

    xy_tens = torch.empty((4, pcloud.shape[0]), device=dev)
    values = torch.tensor([x_min, -1*x_max, y_min, -1*y_max], dtype=dt, device=dev).reshape(-1,1)
    xy_tens[0] = pcloud[:,0]
    xy_tens[1] = pcloud[:,0].mul(-1)
    xy_tens[2] = pcloud[:,1]
    xy_tens[3] = pcloud[:,1].mul(-1)

    xy_bool = torch.empty_like(xy_tens, dtype=torch.bool, device=dev)
    torch.ge(xy_tens, values, out=xy_bool)

    xy_bool[0].logical_and_(xy_bool[1])
    xy_bool[2].logical_and_(xy_bool[3])
    xy_bool[0].logical_and_(xy_bool[2])

    out = torch.tensor(pcloud[xy_bool[0]], dtype=dt, device=dev)
    return out

In [ ]:
arr = torch.ones((5,5), device=dev)[:,0]
arr.mul_(-1)
arr

In [ ]:
#arr = torch.tensor([[-1, 0], [1, 0], [1, 1], [1, 2], [2, 1], [0, 0]])
#get_points_in_pillar(arr, -1, 1, 0, 1)

In [ ]:
arr = torch.rand(10000, 2, dtype=dt, device=dev).mul_(100)
time_method(get_points_in_pillar, runs=100, kwargs={'pcloud': arr, 'x_min': 0, 'x_max': 50, 'y_min': 20, 'y_max': 100})



In [ ]:
#export
def calculate_pillar_bounds(pillars_cfg):
    """
        Calculates the pillar bounds for all pillars
    """
    #pcloud = remove_invalid(pcloud)
    x_min = pillars_cfg.getfloat('x_min')
    y_min = pillars_cfg.getfloat('y_min')
    x_step = pillars_cfg.getfloat('x_step')
    y_step = pillars_cfg.getfloat('y_step')

    # TODO: Specify what happens when n_x and n_y are not exact ints
    n_x = int((pillars_cfg.getfloat('x_max') - x_min) / x_step)
    n_y = int((pillars_cfg.getfloat('y_max') - y_min) / y_step)

    # create tensor containing the pillar indicies in the correct dimension
    x = torch.tensor(range(n_x), dtype=dt, device=dev)
    y = torch.tensor(range(n_y), dtype=dt, device=dev)
    x = x.unsqueeze(0).expand(n_x,n_y)
    y = y.unsqueeze(1).expand(n_x,n_y)
    ind = torch.stack((x,y), dim=2).reshape(-1,2).unsqueeze(1).expand(-1,2,-1).clone()
    ind[:,1].add_(1)

    # calculate the pillar bounds with the indicies tensor
    min_vals = torch.tensor([[x_min, y_min], [x_min, y_min]], dtype = dt, device=dev)
    step_vals = torch.tensor([[x_step, y_step], [x_step, y_step]], dtype=dt, device=dev)
    pillar_bounds = min_vals + ind * step_vals

    return pillar_bounds

##### sample_array()

In [ ]:
#export
def sample_array(arr_list: tuple, length: int):
    """
        samples the array so it has the given length with the same mask
        param arr_list: list of tensors to be sampled the same way
        param length: final length of tensors
    """
    tl = arr_list[0].shape[0]
    mask = torch.zeros(tl, dtype=torch.bool, device=dev)
    mask[:length] = True
    ind = torch.randperm(tl)
    mask = mask[ind]
    out = []
    for i in range(len(arr_list)):
        out.append(arr_list[0][mask])

    return out


In [ ]:

arr = torch.rand(50, 100, 4, dtype=dt, device=dev)
res1, = sample_array((arr,), 5)
res2, = sample_array((arr,), 5)

res1, res1.shape, res2, res2.shape

In [ ]:
arr = torch.rand(23000,100,4, dtype=dt, device=dev)
time_method(sample_array, runs=12000, kwargs={"arr_list": (arr,), "length": 1200})


In [ ]:
#export
def zero_pad_array(arr: torch.tensor, length: int):
    """
        zero pads the array so it has the given length
        param length: final length of array
    """
    lx = arr.shape[0]
    ly = arr.shape[1]
    if arr.dim() == 3:
        lz = arr.shape[2]
        pad_tens = torch.zeros((length-lx, ly, lz), dtype=dt, device=dev)
        out = torch.empty([length, ly, lz], dtype=dt, device=dev)
    else:
        pad_tens = torch.zeros((length-lx, ly), dtype=dt, device=dev)
        out = torch.empty([length, ly], dtype=dt, device=dev)


    return torch.cat((arr, pad_tens), dim=0, out=out)

In [ ]:

arr1 = torch.rand(50, 100, 4, dtype=dt, device=dev)
arr2 = torch.rand(50, 100, dtype=dt, device=dev)
res1 = zero_pad_array(arr1, 100)
res2 = zero_pad_array(arr2, 100)
res1, res1.shape, res2, res2.shape

In [ ]:
arr = torch.rand((100, 100, 4), device=dev)
time_method(zero_pad_array, runs=5000, kwargs={"arr": arr, "length": 1200})
